In [59]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%pip install torchinfo
%restart_python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\haroo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
UsageError: Line magic function `%restart_python` not found.


In [73]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from bacp import BaCPLearner, BaCPTrainer, BaCPTrainingArgumentsLLM
from models import EncoderProjectionNetwork, ClassificationNetwork
from unstructured_pruning import MagnitudePrune, MovementPrune, LocalMagnitudePrune, LocalMovementPrune, WandaPrune, PRUNER_DICT, check_model_sparsity
from LLM_trainer import LLMTrainer, LLMTrainingArguments
from CV_trainer import *
from dataset_utils import get_glue_data
from logger import Logger

import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from tqdm import tqdm
from torchinfo import summary

from datasets.utils.logging import disable_progress_bar
disable_progress_bar()
import os
os.environ["HF_DATASETS_CACHE"] = "/dbfs/hf_datasets"
os.environ["TOKENIZERS_PARALLELISM"] = "false" 

from utils import *
from constants import *

device = get_device()
print(f"{device = }")
BATCH_SIZE_DISTILBERT = 64
NUM_WORKERS = 24


device = 'cpu'


## Baseline Accuracies

In [74]:
# Model initialization
model_name = "vitb16"
model_task = "cifar10"

training_args = CVTrainingArguments(
    model_name=model_name,
    model_task=model_task,
    batch_size=1,
    image_size=224,
    epochs=3,
    learning_type="baseline",
    db=False,
    num_workers=1,
)

trainer = Trainer(training_args=training_args)

if True:
    trainer.train()

# acc = trainer.evaluate()
# print(f"Accuracy = {acc}")



[TRAINER] Initializing model
[TRAINER] Pruning disabled
[TRAINER] Current model sparsity: 0.0
[TRAINER] Initializing data loaders for cifar10
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Saving model checkpoints to ./research/vitb16/cifar10/vitb16_baseline.pt
[LOGGER] Log file created at location: ./log_records\vitb16\baseline\run_7.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.00


Validation Epoch [1/3]:   0%|          | 0/7500 [00:09<?, ?it/s, Accuracy=0.00, Sparsity=0.00]


Epoch [1/3]: Avg Loss: 0.0001 | Avg Accuracy: 0.00 | Model Sparsity: 0.00

[TRAINER] weights saved!


Validation Epoch [2/3]:   0%|          | 0/7500 [00:00<?, ?it/s, Accuracy=0.00, Sparsity=0.00]


Epoch [2/3]: Avg Loss: 0.0001 | Avg Accuracy: 0.00 | Model Sparsity: 0.00



Validation Epoch [3/3]:   0%|          | 0/7500 [00:00<?, ?it/s, Accuracy=0.00, Sparsity=0.00]

Epoch [3/3]: Avg Loss: 0.0001 | Avg Accuracy: 0.00 | Model Sparsity: 0.00



In [65]:
trainer.logger_params

{'model_type': 'cv',
 'model_name': 'vitb16',
 'model_task': 'cifar10',
 'num_classes': 2,
 'embedding_dim': 768,
 'epochs': 3,
 'pruning_epochs': 0,
 'recovery_epochs': 0,
 'batch_size': 64,
 'learning_rate': 2e-05,
 'learning_type': 'baseline',
 'prune': False,
 'target_sparsity': 0,
 'sparsity_scheduler': 'linear',
 'delta_t': 500,
 'enable_mixed_precision': True,
 'device': 'cpu',
 'save_path': './research/vitb16/cifar10/vitb16_baseline.pt',
 'current_sparsity': 0.0}